In [ ]:
from teamUtils import BQ_User # Pricing Analytics package we just installed
from shapely.wkt import loads

In [ ]:
home_project='analytics-askuity-thd' # Make this the project you use in your everyday work, for example Jared would use analytics-pricing-thd
bq = BQ_User(project_name=home_project)

In [ ]:
sql = """
WITH ARRAY_DATA as (
      SELECT 'Division2' as parent_level,
    DIV_NM as parent_level_value ,
    "Buying_Office" AS current_level,
    BYO_NM AS current_level_value,
    BYO_NBR AS current_level_val,
    'Market' as child_level,
     TO_JSON_STRING(ARRAY_AGG(DISTINCT MKT_NM)) AS child_level_value FROM 
      `pr-edw-views-thd.VENDOR_COLLAB_VIEWS.STR_HIER` sth
     
     group by 1,2,3,4,5 ),
  BASE_DATA AS (
  SELECT
    ct.*,
    BYO_NBR AS current_level_val,
    ST_DISTANCE(ST_GEOGPOINT( LNG_NBR,
        LAT_NBR ),
      internal_point_geo ) AS dist
  FROM
    `bigquery-public-data.geo_census_tracts.us_census_tracts_national` AS ct,
    `pr-edw-views-thd.VENDOR_COLLAB_VIEWS.STR_HIER` sth
  WHERE
    sth.LNG_NBR IS NOT NULL
    AND sth.LAT_NBR IS NOT NULL
    AND S_DIV_NM <> 'CLOSED STORES'),
  ORDERED_DIST AS (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY geo_id ORDER BY dist) AS RANK
  FROM
    BASE_DATA )
SELECT
  parent_level,
  parent_level_value,
  current_level,
  current_level_value,
  child_level,
  child_level_value,
  ST_UNION(ST_DUMP(tract_geom,2)) as simplified_blackgroup_geom
FROM (
  SELECT
   *
    
  FROM
    ORDERED_DIST
    LEFT JOIN ARRAY_DATA USING(current_level_val)
  WHERE
    RANK = 1)


"""

In [ ]:
data = bq.SQL.to_df(sql)